<a href="https://colab.research.google.com/github/luoshaoyang/cs224w_final_proj/blob/main/Colab_FinalProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Visible Masking/Watermark Removal with Graph Neural Network

In [2]:
from google.colab import drive
drive.mount('/content/drive')
FOLDERNAME = 'CS224WFinalProj/'
import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))
%cd /content/drive/MyDrive/$FOLDERNAME

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CS224WFinalProj


# Setup
* Import Libraries
* Handy Functions<br>
 1. loadImage
 2. image2NN
 3. NN2Image
 4. plotImage
 5. image2GrayScale
 6. image2Binary
 7. overlayerImg
 8. overlayImg1OverImg2


In [3]:
# Packages
import numpy as np
import pandas as pd
import cv2
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt

import torch
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device

In [4]:
# Functions
def loadImage(path,plotImg=True,gray_scale=False):
  #####################################################
  #Read in image path and returns an image
  #####################################################
  image = cv2.imread(path,flag=int(gray_scale))
  image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY if gray_scale else cv2.COLOR_BGR2RGB)  
  if plotImg:
    plt.figure()
    plt.imshow(image)
    plt.show()
  return image

def image2Tensor(img,gray_scale=True):
  #####################################################
  #Read in image, and return tensor of the image
  #  more interesting reading: https://towardsdatascience.com/image-read-and-resize-with-opencv-tensorflow-and-pil-3e0f29b992be
  #####################################################
  # convert BGR image to RGB image
  img = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY if gray_scale else cv2.COLOR_BGR2RGB)
  transform = transforms.Compose([transforms.ToTensor()])
  img_tensor = transform(img)
  return img_tensor

def Tensor2Image(img_tensor,dim_order=None,gray_scale=True,plotImg=True):
  #####################################################
  #Read in tensor, and return plotable image
  #####################################################
  img = img_tensor.numpy()
  if dim_order:
    img = np.transpose(img,order)
  img = cvtColor(img,cv2.COLOR_BGR2GRAY if gray_scale else cv2.COLOR_BGR2RGB)
  if plotImg:
    plt.figure()
    plt.imshow(image)
    plt.show()
  return img

def plotImg(image):
  plt.figure()
  plt.imshow(image)
  plt.show()

def image2GrayScale(image):
  img = cvtColor(img,cv2.COLOR_BGR2GRAY)
  return img

def image2Binary(image):
  img = image2GrayScale(image)
  img = np.round(img)
  return img

def overlayerImg(size,rgb):
  #####################################################
  #Create a box overlay image that could go above the license plate
  #  rgb being a vector of shape (,3); size being a vector of shape (row,col,3)
  #####################################################
  img = np.ndarray([[[rgb[0]]*size.shape[0]]*size.shape[1],
                    [[rgb[1]]*size.shape[0]]*size.shape[1],
                    [[rgb[2]]*size.shape[0]]*size.shape[1]])
  return img

def overlayImg1OverImg2(img1,img2):
  #####################################################
  #Overlay the box image (img1) above the license plate(img2)
  #  center the images at the same place
  #####################################################
  img = img2.copy()
  y_lower = np.round(img.shape[0]/2)-np.round(img1.shape[0]/2)
  y_upper = y_lower+img1.shape[0]# (y_lower,y_upper]
  img[y_lower:y_upper,:,:] = img1
  return img

#Data Processing
* Read in the datasets
* Create masked/watermarked datasets
* Split for training and testing

In [ ]:
# Load DataSet
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

idx=0#for image 0 in training
path = os.path.join('../data', train.loc[idx, 'Label'], train.loc[idx, 'Image'])
loadImage(path)

In [ ]:
# Create masked datasets

In [ ]:
# Split data for training and testing

# Model Build
* Learn GNN embedding
* Apply GCN to predict<br>
  1. GCN training
  2. Accuracy measurement
* Apply label propagation to predict<br>
  1. Edge weight training
  2. propagation iteration
  3. Accuracy measurement

In [ ]:
# GNN embedding learning

In [ ]:
# GCN Model

In [ ]:
# Label Propagation
#####################################
#prepare edge_weight and nn graph
#- dict edge_weights = {(from_node_id,to_node_id):weight}
edge_weights = {
    (1,2):1,(2,1):1,
    (2,3):1,(3,2):1,
    (3,4):1,(4,3):1,
    (4,5):1,(5,4):1,
    (5,1):1,(1,5):1
}
#- list edges = [(from_node_id,to_node_id)]
edges = edge_weights.keys()
#- dict node_featuress = {node_id:[r,g,b]}
node_features = {
    1:np.array([1,0,0]),
    2:np.array([0,1,0]),
    3:np.array([0,0,1]),
    4:np.array([1,1,0]),
    5:np.array([1,1,1])
}
#- dict node_degree_in = {node_id:int}
#- dict node_degree_out = {node_id:int}
node_degree_in = {}
node_degree_out = {}
for relation in edge_weights.keys():
  if relation[0] in node_degree_out.keys():
    node_degree_out[relation[0]] += 1
  else:
    node_degree_out[relation[0]] = 1
  if relation[1] in node_degree_out.keys():
    node_degree_in[relation[1]] += 1
  else:
    node_degree_in[relation[1]] = 1
#- dict mask_label = {node_id:1/0}
mask_lable = {
    1:0,
    2:1,
    3:0,
    4:1,
    5:0
}

#####################################
#vanilla label propagation
epsilon = 0.01
max_iter = 10000
#function to calculate node feature differences
def featureDiff(iter_features,curr_features,agg=max):
  assert iter_features.keys() == curr_features.keys()
  for node in iter_features.keys():
    diff_dict[node] = agg(curr_features[node]-iter_features[node])
  return diff_dict

iter_features = node_features.copy()
for i in range(max_iter):
  curr_features = iter_features.copy()
  #update label for masked nodes only
  for relation in edge_weights.keys():
    if mask_lable[relation[1]]:
      curr_feature[relation[1]] += iter_feature[relation[0]]*edge_weights[relation]/len(node_degree_in[relation[1]])
  #measure difference
  errors = featureDiff(iter_features,curr_features)
  iter_features = curr_features.copy()
  if max(errors.values())<epsilon:
    break

#####################################
#correct & smooth
epsilon = 0.01
max_iter = 10000
alpha = 0.1
diffusion_scale = 0.1
#[ToDo]function to calculate diffusion matrix
def adjDiffuse(adjacency):
  diffusion = adjacency.copy()
  return diffusion
#[ToDo]function to calculate correction
def correctStep(adjacency,errors,alpha,max_iter=max_iter,epsilon=epsilon,agg=max):
  #[ToDo]dimension assersion
  assert True
  diffusion = adjDiffuse(adjacency)
  e = errors.copy()
  for i in range(max_iter):
    curr_e = e.copy()
    curr_e = (1-alpha)*e+alpha*diffusion.dot(e)
    if agg(curr_e-e)<=epsilon:
      break
  return correction
#[ToDo]function to do smooth step
def smoothStep(adjacency,errors,alpha,max_iter=max_iter,epsilon=epsilon,agg=max):
  #[ToDo]dimension assersion
  assert True
  diffusion = adjDiffuse(adjacency)
  smooth = errors.copy()
  for i in range(max_iter):
    curr_z = z.copy()
    curr_z = (1-alpha)*z+alpha*diffusion.dot(z)
    if agg(curr_z-z)<=epsilon:
      break
  return smooth

iter_features = node_features.copy()
#[ToDo]correct step
correction = correctStep()
iter_features += diffusion_scale*correction
#[ToDo]smooth step
iter_features = smoothStep()
